In [183]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [198]:
%autoreload

In [200]:
from santautils import *

In [5]:
image = df_to_image(pd.read_csv("data/image.csv"))

with open("output/web_renders/test_path.json") as f:
    path = json.load(f)

len(path)

In [7]:
duplicate_ids = [i+1 for i, (config, next_config) in enumerate(zip(path, path[1:])) if config == next_config]
len(duplicate_ids)

725

In [20]:
deduped_path = [config for i, config in enumerate(path) if i not in duplicate_ids]

In [20]:
def config_to_string(config):
    return ';'.join([' '.join(map(str, vector)) for vector in config])


submission = pd.Series(
    [config_to_string(config) for config in deduped_path],
    name="configuration",
)

submission.head()

0    64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 0
1    64 0;-32 0;-16 0;-8 0;-4 0;-2 0;-1 0;-1 1
2     64 0;-32 0;-16 1;-8 0;-4 0;-2 0;-1 0;0 1
3    64 -1;-32 0;-16 1;-8 0;-4 0;-2 0;-1 0;1 1
4    64 -1;-32 0;-16 2;-8 0;-4 0;-2 0;-1 1;0 1
Name: configuration, dtype: object

In [22]:
submission.to_csv("output/submissions/pycharm_first.csv", index=False)

In [26]:

step_costs = [step_cost(config, next_config, image) for config, next_config in zip(path, path[1:])]
len(step_costs)

74351

In [120]:
def path_to_df(path):
    position = [get_position(config) for config in path]
    reconf_costs = [None] + [reconfiguration_cost(config, next_config) 
                                                 for config, next_config 
                                                 in zip(path, path[1:])]
    step_costs = [None] + [step_cost(config, next_config, image) 
                                      for config, next_config 
                                  in zip(path, path[1:])]
    return pd.DataFrame({"config": path,
                         "position": position,
                         "reconf_cost": reconf_costs,
                         "step_cost": step_costs,
                        })

In [95]:
def string_to_config(row_string):
    a = (pair.split() for pair in row_string.split(";"))
    b = ([int(x), int(y)] for (x, y) in a)
    return list(b)

In [122]:
kaggle_submission = pd.read_csv("output/submissions/submission_online.csv").astype("string")
kaggle_submission.head()

online_path = [string_to_config(row) for row in kaggle_submission["configuration"]]

In [128]:
with open("output/web_renders/online_sub.json", "w") as file:
    file.write(json.dumps(online_path))

In [126]:
online_df = path_to_df(online_path)
online_df.head()

,config,position,reconf_cost,step_cost
0,"[[64, 0], [-32, 0], [-16, 0], [-8, 0], [-4, 0]...","(0, 0)",NaN,NaN
1,"[[64, -1], [-32, 0], [-16, 0], [-8, 0], [-4, 0...","(0, -1)",1.0,1.094118
2,"[[64, -2], [-32, 0], [-16, 0], [-8, 0], [-4, 0...","(0, -2)",1.0,1.176471
3,"[[64, -3], [-32, 0], [-16, 0], [-8, 0], [-4, 0...","(0, -3)",1.0,1.188235
4,"[[64, -4], [-32, 0], [-16, 0], [-8, 0], [-4, 0...","(0, -4)",1.0,1.200000


In [125]:
local_df = path_to_df(deduped_path)
local_df.head()

,config,position,reconf_cost,step_cost
0,"[[64, 0], [-32, 0], [-16, 0], [-8, 0], [-4, 0]...","(0, 0)",NaN,NaN
1,"[[64, 0], [-32, 0], [-16, 0], [-8, 0], [-4, 0]...","(0, 1)",1.000000,1.105882
2,"[[64, 0], [-32, 0], [-16, 1], [-8, 0], [-4, 0]...","(1, 2)",1.414214,1.473037
3,"[[64, -1], [-32, 0], [-16, 1], [-8, 0], [-4, 0...","(2, 1)",1.414214,1.590684
4,"[[64, -1], [-32, 0], [-16, 2], [-8, 0], [-4, 0...","(1, 3)",1.732051,2.096757


In [148]:
image = df_to_image(pd.read_csv("data/image.csv"))
config = [(64, 0), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 0)]
n = config[0][0] * 2
points = list(product(range(-n, n + 1), repeat=2))
unvisited = set(points)

cheapest_local = get_cheapest_unvisited_neighbor(config, unvisited, partial(step_cost, image=image))
cheapest_local

[(64, -1), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 1)]

In [147]:
cheapest_online = online_path[1]
cheapest_online

[[64, -1], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]

In [201]:
for candidate in [cheapest_local, cheapest_online]:
    print(f"{candidate=}")
    for cost in [reconfiguration_cost, color_cost_from_config, step_cost]:
        try:
            result = cost(config, candidate)
        except TypeError:
            result = cost(config, candidate, image)
        display(result)

candidate=[(64, -1), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 1)]


1.4142135623730951

0.0

1.4142135623730951

candidate=[[64, -1], [-32, 0], [-16, 0], [-8, 0], [-4, 0], [-2, 0], [-1, 0], [-1, 0]]


1.0

0.09411764705882375

1.0941176470588236

In [139]:
sorted(get_neighbors_positions_costs(config, image), key=lambda x: x[-1])

[([(64, -1), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, -1)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, -1), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 0), (-16, 0), (-8, 0), (-4, 0), (-2, -1), (-1, 0), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 0), (-16, 0), (-8, 0), (-4, -1), (-2, 0), (-1, 0), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 0), (-16, 0), (-8, -1), (-4, 0), (-2, 0), (-1, 0), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 0), (-16, -1), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, -1), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 0)],
  (0, -1),
  1.0941176470588236),
 ([(64, 0), (-32, 1), (-16, 0), (-8, 0), (-4, 0), (-2, 0), (-1, 0), (-1, 0)],
  (0, 1),
  1.1058

In [203]:
(3**8)**2

43046721

In [204]:
5**8

390625

In [205]:
7**8

5764801